In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime, date

In [3]:
# Tải lên dữ liệu Customer Address từ file excel

cust_address = pd.read_excel('Raw_data.xlsx', sheet_name='CustomerAddress')

In [4]:
# Kiểm tra 5 bản ghi đầu tiên từ dữ liệu Customer Address

cust_address.head()

,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,New South Wales,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9


In [5]:
# Thông tin về các cột và kiểu dữ liệu của dữ liệu Customer Address

cust_address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         3999 non-null   int64 
 1   address             3999 non-null   object
 2   postcode            3999 non-null   int64 
 3   state               3999 non-null   object
 4   country             3999 non-null   object
 5   property_valuation  3999 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 187.6+ KB


Chúng ta có thể thấy rằng dữ liệu của các cột trông có vẻ ổn.

## Tổng số bản ghi

In [6]:
# Tổng số hàng và cột trong tập dữ liệu

print('Total records (row) in the dataset: {}'.format(cust_address.shape[0]))
print('Total columns in the dataset: {}'.format(cust_address.shape[1]))

Total records (row) in the dataset: 3999
Total columns in the dataset: 6


## Các cột số và các cột không phải số

In [7]:
# Lấy ra các cột số
df_numeric = cust_address.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values
print("The numeric columns are :")
print(numeric_cols)

# Lấy ra các cột không phải số
df_non_numeric = cust_address.select_dtypes(exclude=[np.number])
non_numeric_cols = df_non_numeric.columns.values
print("The non-numeric columns are :")
print(non_numeric_cols)

The numeric columns are :
['customer_id' 'postcode' 'property_valuation']
The non-numeric columns are :
['address' 'state' 'country']


## 1. Kiểm tra giá trị bị thiếu

Kiểm tra xem tập dữ liệu có tồn tại bất kỳ giá trị thiếu nào không. Nếu giá trị thiếu xuất hiện cho một cột cụ thể, tùy thuộc vào tình huống thì: 
- Cột đó có thể bị loại bỏ (trong trường hợp một số lượng lớn dữ liệu bị thiếu) 
- Có thể loại bỏ giá trị thiếu đó (trong trường hợp một số lượng ít dữ liệu bị thiếu)
- Một giá trị phù hợp sẽ được điền vào cột có giá trị thiếu

In [8]:
# Tổng số giá trị bị thiếu

cust_address.isnull().sum()

customer_id           0
address               0
postcode              0
state                 0
country               0
property_valuation    0
dtype: int64

<b>Trong tập dữ liệu không có giá trị nào bị thiếu.</b>

## 2. Kiểm tra sự không nhất quán trong dữ liệu.

Chúng ta sẽ kiểm tra xem liệu có dữ liệu không nhất quán hoặc dữ liệu nhập sai / lỗi chính tả trong các cột phân loại.</br>
Các cột cần được kiểm tra là: <b>'address', 'postcode', 'state', 'country'.</b>

### 2.1. Cột State

In [9]:
cust_address['state'].value_counts()

state
NSW                2054
VIC                 939
QLD                 838
New South Wales      86
Victoria             82
Name: count, dtype: int64

Có dữ liệu không nhất quán trong cột 'State'. Đối với New South Wales và Victoria, chúng ta có hai giá trị, một là tên đầy đủ và một là tên viết tắt của chúng. Tên các tiểu bang cần được chuẩn hóa và các cột với tên bang là <b>New South Wales sẽ được thay thế bằng NSW</b> và các cột với tên bang là <b>Victoria sẽ được thay thế bằng VIC</b>.

In [10]:
# Hàm thay thế tên đầy đủ của các tiểu bang thành các tên viết tắt của chúng.

def replace_state_names(state_name):
    if state_name == 'New South Wales':
        return 'NSW'
    elif state_name == 'Victoria':
        return 'VIC'
    else:
        return state_name
    
# Áp dụng hàm trên vào cột "state"
cust_address['state'] = cust_address['state'].apply(replace_state_names)

Sau khi áp dụng hàm trên, tên của tiểu bang đã được chuẩn hóa và không còn sự không nhất quán nào trong cột "state" nữa.

In [11]:
cust_address['state'].value_counts().reset_index()

,state,count
0,NSW,2140
1,VIC,1021
2,QLD,838


### 2.2. Cột Country

Không có <b>dữ liệu không nhất quán</b> trong cột <b>country</b>

In [12]:
cust_address['country'].value_counts().reset_index()

,country,count
0,Australia,3999


### 2.3. Cột Postcode

Không có <b>dữ liệu không nhất quán</b> trong cột <b>postcode</b>

In [13]:
cust_address[['address', 'postcode', 'state', 'country']].drop_duplicates()

,address,postcode,state,country
0,060 Morning Avenue,2016,NSW,Australia
1,6 Meadow Vale Court,2153,NSW,Australia
2,0 Holy Cross Court,4211,QLD,Australia
3,17979 Del Mar Point,2448,NSW,Australia
4,9 Oakridge Court,3216,VIC,Australia
...,...,...,...,...
3994,1482 Hauk Trail,3064,VIC,Australia
3995,57042 Village Green Point,4511,QLD,Australia
3996,87 Crescent Oaks Alley,2756,NSW,Australia
3997,8194 Lien Street,4032,QLD,Australia


## 3. Kiểm tra trùng lặp

Chúng ta cần đảm bảo rằng không có sự trùng lặp của bản ghi trong tập dữ liệu. Điều này có thể dẫn đến lỗi trong phân tích dữ liệu do chất lượng dữ liệu kém. Nếu có các hàng dữ liệu trùng lặp, chúng ta cần loại bỏ những bản ghi đó.</br>
Để kiểm tra sự trùng lặp của các bản ghi, trước tiên chúng ta cần loại bỏ cột khóa chính của tập dữ liệu, sau đó áp dụng hàm drop_duplicates() được cung cấp bởi Python.

In [14]:
# Xóa cột khóa chính, tức là customer_id, và lưu vào một dataframe tạm thời.
cust_address_dedupped = cust_address.drop('customer_id', axis=1).drop_duplicates()

print("Number of records after removing customer_id (pk), duplicates : {}".format(cust_address_dedupped.shape[0]))
print("Number of records in original dataset : {}".format(cust_address.shape[0]))

Number of records after removing customer_id (pk), duplicates : 3999
Number of records in original dataset : 3999


<b>Vì cả hai giá trị đều giống nhau, nên không có bản ghi trùng lặp trong tập dữ liệu.</b>

## 4. Xuất tập dữ liệu Customer Address đã được làm sạch sang định dạng csv

Tập dữ liệu Customer Address đã được làm sạch. Do đó, chúng ta có thể xuất dữ liệu sang định dạng CSV để tiếp tục phân tích dữ liệu của phân khúc khách hàng bằng cách kết hợp nó với các bảng khác.

In [15]:
cust_address.to_csv('CustomerAddress_Cleaned.csv', index=False)

## 5. Kiểm tra số lượng bản ghi ở tập dữ liệu CustomerDemographic_Cleaned

Kiểm tra với tập dữ liệu (CustomerDemographic_Cleaned.csv), là nơi chứa tất cả các dữ liệu chính về khách hàng bao gồm cả Customer ID của tập dữ liệu Customer Address. Nên khi chúng ta loại bỏ những Customer ID trong tập dữ liệu Customer Demographic do bị thiếu ngày sinh và ngoại lệ, <b>sẽ dẫn đến vấn đề dữ liệu ở 2 tập dữ liệu không còn liên kết và tương thích với nhau nữa.</b>

In [16]:
cust_demo_detail = pd.read_csv('CustomerDemographic_Cleaned.csv')

In [17]:
cust_demo_detail.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,Age
0,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,70
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,43
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,15.0,70
3,4,Talbot,NaN,Male,33,1961-10-03,Missing,IT,Mass Customer,N,No,7.0,62
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,Missing,Affluent Customer,N,Yes,8.0,46


In [18]:
# Số lượng bản ghi bị chênh lệch giữa 2 tập dữ liệu sau khi làm sạch

print("Total Records in Customer_Demographic_Table : {}".format(cust_demo_detail.shape[0]))
print("Total Records in Customer_Address_Table : {}".format(cust_address.shape[0]))
print('In Demographic Table {} records are getting dropped due to data cleaning process in Demographic Table'
      .format(cust_address.shape[0]-cust_demo_detail.shape[0]))

Total Records in Customer_Demographic_Table : 3912
Total Records in Customer_Address_Table : 3999
In Demographic Table 87 records are getting dropped due to data cleaning process in Demographic Table


In [27]:
cust_drop = cust_address.merge(cust_demo_detail , left_on = 'customer_id', right_on='customer_id'
                    , how='inner')
cust_drop.head()

,customer_id,address,postcode,state,country,property_valuation,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,Age
0,1,060 Morning Avenue,2016,NSW,Australia,10,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,70
1,2,6 Meadow Vale Court,2153,NSW,Australia,10,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,43
2,4,0 Holy Cross Court,4211,QLD,Australia,9,Talbot,NaN,Male,33,1961-10-03,Missing,IT,Mass Customer,N,No,7.0,62
3,5,17979 Del Mar Point,2448,NSW,Australia,4,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,Missing,Affluent Customer,N,Yes,8.0,46
4,6,9 Oakridge Court,3216,VIC,Australia,9,Curr,Duckhouse,Male,35,1966-09-16,Missing,Retail,High Net Worth,N,Yes,13.0,57
